## Open Session Hive

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os
import requests
from datetime import datetime
from urllib.request import urlopen

## Hive Context

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '2g'),
     ("spark.driver.allowMultipleContexts", "true"),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
sparkHive = SparkSession.builder \
        .master("local[*]") \
        .appName("testreplace") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

## อ่านข้อมูลจาก Hive และส่งออกไปยัง CSV

In [ ]:
hiveDF = sparkHive.read.table('pyspark.affairs')
hiveDF.write.mode('overwrite').option("header","true").csv("file:///tmp/affairs.csv")

## ปิด Session Hive

In [ ]:
sparkHive.stop()

## Open Session MySQL

In [ ]:
sparkMysql = SparkSession.builder \
           .appName('MySQL') \
           .config("spark.jars", "./jars/mysql-connector-java-8.0.32.jar") \
           .getOrCreate()

## Schema

In [ ]:
cols = StructType([
    StructField("affairs", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("age", StringType(), True),
    StructField("yearsmarried", StringType(), True),
    StructField("children", StringType(), True),
    StructField("religiousness", StringType(), True),
    StructField("education", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("rating", StringType(), True),
])

## อ่านข้อมูจาก CSV (ใน Session Hive) และเขียนลง MySQL

In [ ]:
df = sparkMysql.read.csv("file:///tmp/affairs.csv", schema=cols)
df.write \
  .format("jdbc") \
  .mode('overwrite') \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://192.168.10.22:3306/test") \
  .option("dbtable", "affairs") \
  .option("user", "root") \
  .option("password", "P@ssw0rd") \
  .save()

## ปิด Session MySQL

In [ ]:
sparkMysql.stop()